# Coding your own RNN

Using this pre-filled notebook, we will code our own RNN for sentence classification. For now, we'll keep using IMDB, as the goal of this part is to understand how an RNN works.

Unlike our previous lab, we will also learn the embedding layer. Which means we need to deal with vocabulary by ourselves.

In [1]:
from functools import partial
from typing import Callable, Dict, Generator, List, Tuple

from datasets import load_dataset
import numpy as np
from sklearn.utils import shuffle
import torch
from torch import nn
from torchtext.vocab import vocab, Vocab
from torchtext.data.utils import get_tokenizer

from tqdm.auto import tqdm

/home/aeschylli/scia/nlp/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Dataset
We load the dataset and split the training set in a stratified train/validation set.

In [2]:
dataset = load_dataset("imdb")
train_dataset = dataset["train"].train_test_split(
    stratify_by_column="label", test_size=0.2, seed=42
)
test_df = dataset["test"]
train_df = train_dataset["train"]
valid_df = train_dataset["test"]
train_df.shape, valid_df.shape, test_df.shape

Found cached dataset imdb (/home/aeschylli/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
100%|██████████| 3/3 [00:00<00:00, 29.03it/s]
Loading cached split indices for dataset at /home/aeschylli/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-5f37fd0866e4f89f.arrow and /home/aeschylli/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-dd5732a0e6ac784c.arrow


((20000, 2), (5000, 2), (25000, 2))

## Vocabulary (1 point)

**\[1 point\]** Build your own vocabulary. The [example provided in torchtext documentation](https://pytorch.org/text/stable/vocab.html#id1) might be of help.
* Don't forge to setup the `min_freq` parameter to not include unfrequent noise.
* You will need a tokenizer. Reuse the `basic_english` one from the our previous lab.
* For an RNN we need two special tokens: `<unk>`, for unknown words, and `<pad>` for padding.

In [11]:
from collections import Counter, OrderedDict
tokenizer = get_tokenizer("basic_english", language="en")

words = [word for sentence in train_df['text'] for word in tokenizer(sentence)]
counter = Counter(words)
sorted_by_freq_tuples = sorted(counter.items(), key=lambda x: x[1], reverse=True)
ordered_dict = OrderedDict(sorted_by_freq_tuples)
unk_token = '<unk>'
pad_token = '<pad>'
default_index = -1
vocabulary = vocab(ordered_dict=ordered_dict, specials=[unk_token, pad_token], min_freq=3)
vocabulary.set_default_index(default_index)
vocabulary.set_default_index(vocabulary[unk_token])

## Vectorize and batch the input (3 points)

As seen in class, our model should take one-hot encoded vectors corresponding to the each token vocabulary id. However, computing a vector x matrix multiplication for every input is unnecessarily costly. Multiplying a one-hot vector with a matrix is the equivalent of taking one row of the matrix. In pyTorch, we provide ids for each token which will be used as input to an `nn.Embedding` layer. The id is simply the row in the embedding matrix.

**\[1 point\]** Fill the `vectorize_text` function returning a 1D torch tensor of `torch.long` for each input text.

In [17]:
def vectorize_text(
    text: str, vocabulary: Vocab, tokenizer: Callable[[str], List[str]]
) -> torch.Tensor:
    """
    Generate a tensor of vocabluary IDs for a given text.
    Args:
        text: the input text.
        vocabulary: a Vocab objects.
        tokenizer: a text tokenizer.
    Returns:
        A tensor of IDs (torch.long).
    """
    tokens = tokenizer(text)
    res = []
    for token in tokens:
        res.append(vocabulary[token])
    return torch.Tensor(res)

In [18]:
text_pipeline = partial(vectorize_text, vocabulary=vocabulary, tokenizer=tokenizer)

Check the function is working correctly, especially it should return the right special id for unknown words.

In [19]:
text_pipeline("Some text I am thinking about... ragafqfa")

tensor([5.6000e+01, 3.1600e+03, 1.3000e+01, 2.4400e+02, 5.2600e+02, 5.0000e+01,
        3.0000e+00, 3.0000e+00, 3.0000e+00, 0.0000e+00])

In [20]:
X_train = [text_pipeline(text) for text in tqdm(train_df["text"])]
y_train = train_df["label"]
X_valid = [text_pipeline(text) for text in tqdm(valid_df["text"])]
y_valid = valid_df["label"]
X_test = [text_pipeline(text) for text in tqdm(test_df["text"])]
y_test = test_df["label"]

100%|██████████| 25000/25000 [00:16<00:00, 1549.37it/s]


In [35]:
X_train[1].size()[0]
X_train[1].add_(150)
X_train[1]

tensor([  158.,   184.,  1400.,   189.,   275.,   168.,   232.,   168.,   189.,
          254.,   170.,   156.,   601.,   157.,   851.,   150.,   155.,   150.,
          154.,   172.,   152.,   566.,   167.,   210.,  1085.,   229.,   150.,
          157.,   150.,   155.,   152.,  1802.,   167.,   156.,   270.,   326.,
         2561.,   306.,   153.,   428.,   201.,   206.,   157.,   152.,   312.,
          228.,   179.,   194.,  1104.,   150.,   203.,   157.,   150.,   161.,
          215.,   183.,   250.,   152.,   173.,   156.,   270.,   283.,   255.,
          153.,   153.,   158.,   290.,   164.,   167.,   533.,   160.,   158.,
          242.,   164.,   173.,   156.,   150.,   154.,   201.,   176.,   335.,
          158.,   224.,   299.,   165.,   160.,   637., 12184.,   154.,   176.,
         3554.,  1001.,   155.,   596.,   156.,   530.,   157.,   759.,   157.,
          286.,   276.,   305.,   159.,   150.,   159.,   154.,   165.,   160.,
        10747.,   491., 16244.,   533., 

To speed up the training process, we turn the inputs into batches, as we did last time. For batches to work, every line must have the same lengths. Last time, it was implicit as only a vector (the average of all embeddings) was provided. This time, every line has the length of a different review.

To go around this problem, we use padding. So every line within a batch is padded to the length of its longest element.

* **\[1 point\]** Fill the data generator function.
* **\[1 point\]** On which side should you pad and why?

In [33]:
def data_generator(
    X: List[torch.tensor], y: List[int], pad_id: int, batch_size: int = 32
) -> Generator[Tuple[torch.Tensor, torch.Tensor], None, None]:
    """
    Yield batches from given input data and labels.
    Args:
        X: a list of tensor (input features).
        y: the corresponding labels.
        batch_size: the size of every batch [32].
    Returns:
        A tuple of tensors (features, labels).
    """
    X, y = shuffle(X, y)
    for i in range(0, len(X), batch_size):
        max_length = 0
        for j in range(i):
            if X[j].size() > max_length:
                max_length = X[j].size()
        for j in range(i):
            
        yield (torch.stack(X[i:i+batch_size]), torch.tensor(y[i:i+batch_size]))

IndentationError: expected an indented block after 'for' statement on line 19 (521286564.py, line 21)

In [34]:
train_gen = lambda: data_generator(X_train, y_train, vocabulary[pad_token])
valid_gen = lambda: data_generator(X_valid, y_valid, vocabulary[pad_token])
test_gen = lambda: data_generator(X_test, y_test, vocabulary[pad_token])

## Classifier (3 points)

**\[3 points\]** Code your own RNN. Fill the `RNN` class correctly. Remember, an RNN has 3 elements.
* An embedding layer to turn one-hot vectors into dense vectors.
* A linear layer which takes the concatenation of the current embedding and hidden layer and send it to the hidden layer of our next step.
  * `embedding_size + hidden_size -> hidden_size`
* A linear layer which takes the concatenation of the current embedding and hidden layer and send it to the output.
  * `embedding_size + hidden_size -> 1`
* The provided `init_hidden` function sho

## Training (2 points)

Training is a bit different than usual. We will need to sequentially (but in "batch parallel") go through an input, keeping track of the hidden layer, and use the last output as prediction.

**\[2 point\]** Code the training loop.
* Note that for each batch, you need to loop through the whole input and use the output of the last token as input to your criterion.
* Keep the best model evaluated on the validation set.
* Plot the training and validation losses.
* Training will take some time (~30 min on a T4 GPU). Make sure your results appear in the notebook.

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

In [ ]:
n_embedding = 32
n_hidden = 64
model = RNN(len(vocabulary.get_itos()), n_embedding, n_hidden).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

## Evaluation (1 point)

* **\[1 point\]** Compute the accuracy for all 3 splits.